Installed Langchain:

pip install -qU "langchain[openai]"

pip install -U langchain-community

pip install pypdf

pip install sentence-transformers

pip install hf_xet

In [11]:
import tqdm as notebook_tqdm

In [12]:
#Load the LargeLanguageModel

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

from transformers import AutoConfig

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

config = AutoConfig.from_pretrained("microsoft/phi-2")

# 4. Load model with offloading enabled
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    device_map="auto",  # OR pass custom map below if needed
    quantization_config=bnb_config,
    offload_folder="./offload",  # Where to store offloaded layers
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.13s/it]


In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

from transformers import AutoConfig

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

config = AutoConfig.from_pretrained("microsoft/phi-2")

# 4. Load model with offloading enabled
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    device_map="auto",  # OR pass custom map below if needed
    quantization_config=bnb_config,
    offload_folder="./offload",  # Where to store offloaded layers
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.02s/it]


In [14]:
from langchain.document_loaders import PyPDFLoader

# Load the PDF file
pdf_loader = PyPDFLoader("trainingdata.pdf")
documents = pdf_loader.load()

# Display the loaded documents
print(documents)

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-05-05T18:52:22-05:00', 'author': 'Karteek varma Pericharla', 'moddate': '2025-05-05T18:52:22-05:00', 'source': 'trainingdata.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content="Biographical Background: \nKarteek Varma Pericharla, who is in short called Karteek and sometimes Varma, is a dedicated and \nambitious Artificial Intelligence student who pursued his master’s in computer science with a \nspecialization in Artificial Intelligence at Illinois Institute of Technology (IIT), Chicago and this is \nwhere he is currently at. He is expected to graduate in May 2025. This marks his second master's \ndegree, building upon his prior MBA in Business Analytics, Artificial Intelligence, and Machine \nLearning from Woxsen University, where he learned application of AI for business at Hyderabad. His \nfascination with AI was sparked during h

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split the documents into chunks
chunks = text_splitter.split_documents(documents)

# Display the chunks
print(chunks)

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-05-05T18:52:22-05:00', 'author': 'Karteek varma Pericharla', 'moddate': '2025-05-05T18:52:22-05:00', 'source': 'trainingdata.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content="Biographical Background: \nKarteek Varma Pericharla, who is in short called Karteek and sometimes Varma, is a dedicated and \nambitious Artificial Intelligence student who pursued his master’s in computer science with a \nspecialization in Artificial Intelligence at Illinois Institute of Technology (IIT), Chicago and this is \nwhere he is currently at. He is expected to graduate in May 2025. This marks his second master's \ndegree, building upon his prior MBA in Business Analytics, Artificial Intelligence, and Machine \nLearning from Woxsen University, where he learned application of AI for business at Hyderabad. His \nfascination with AI was sparked during h

In [16]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = Chroma.from_documents(chunks, embedding=embedding_model, persist_directory="C:\\Users\\varma\\my-site\\backend\\chroma_store", collection_name="karteek_context")

db.persist()

The cells below is only for test inference, and do not need once deployed unless to test

In [17]:
query = "Tell me about his projects"
retrieved_docs = db.similarity_search(query, k=2)

#Get the context as plain text
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

print("Context: ", context)


Context:  • His skills in AI/ML include understanding and the ability to build and utilize  Neural 
networks, Clustering, Classification, Regression, Transformer models, Reinforcement 
Learning (DDQL) algorithms 
• He is keen at NLP and has worked on projects that include utilizing Sentiment analysis, Text 
summarization, GPT fine-tuning, BLEU/ROUGE/Perplexity metrics, Word2Vec, TextBlob, 
NLTK 
• To complement his abilities as a ML engineer he also learned to  Visualize data where 
he learned to used tools like Tableau, Power BI, Matplotlib, Seaborn, ggplot 
• He has also learned some Data Engineering skills involving: AWS pipelines, graph-based 
processing and feature engineering 
• He also possess skills in Computer Vision and can perform tasks like Object detection, 
facial recognition, real-time video analysis and much more 
• Tools: Git, CI/CD pipelines, Power Vision (for HMI) 
 
Karteek has implemented a multiple of Professional, Personal and Academics projects, which 
are:

• H

In [18]:
collections = db._client.list_collections()
print([c.name for c in collections])


['karteek_context']


In [19]:
# prompt = f"""Context:
# {context}
#
# Answer the following question strictly using the context.
#
# Question: {query}
# Answer:"""

In [20]:
# input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

# with torch.no_grad():
#     output_ids = model.generate(
#         input_ids,
#         max_new_tokens=150,
#         pad_token_id=tokenizer.eos_token_id,
#         eos_token_id=tokenizer.eos_token_id,
#     )

# Slice to get only the generated part (excluding prompt)
# generated_tokens = output_ids[0][input_ids.shape[-1]:]

# Decode only generated tokens
# response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
# response = response.split("Exercise")[0].strip()
# response = response.split("Question")[0].strip()

# print(response)
